In [3]:
from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.utils.measurement.synthetic_functions import hartmann6
from ax.utils.notebook.plotting import init_notebook_plotting, render

import numpy as np

import requests

Mock 'driver'/ ax service client. Call the 'launch new experiment' endpoint, then query the /update campaign endpoint with new data 

In [106]:
def evaluate(parameterization):
    x = np.array([parameterization.get(f"x{i+1}") for i in range(6)])
    return {"hartmann6": (hartmann6(x), 0.0)}

In [87]:
num_init_trials = 12

init_trials = []

for i in range(num_init_trials):
    init_x = np.random.rand(6)
    init_parameterization = {f"x{i+1}":xval for i, xval in enumerate(init_x)}
    init_data = {"hartmann6": (hartmann6(init_x), 0.0)}
    init_trial = {"parameterization":init_parameterization, "results":init_data}
    init_trials.append(init_trial)

In [92]:
# 1. set new experiment

response = requests.post('http://127.0.0.1:5000/new_experiment')

In [93]:
response.content

b'Initialized new experiment'

In [94]:
init_trials[0]['results'].keys()

dict_keys(['hartmann6'])

In [95]:
r = requests.post('http://127.0.0.1:5000/append_data', json = init_trials)

In [99]:
r = requests.get('http://127.0.0.1:5000/check_trials')

In [101]:
r.content

b'Current trials are [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]'

In [111]:
trial_data = requests.get('http://127.0.0.1:5000/get_next_trial')

In [112]:
trial_data

<Response [200]>

In [113]:
trial_data.json()

{'parameterization': {'x1': 0.13046890191068267,
  'x2': 0.3523424846459916,
  'x3': 0.37606535692064263,
  'x4': 0.007228339100169378,
  'x5': 0.6585345341310248,
  'x6': 0.8117258055061629},
 'trial_index': 13}

In [102]:
parameterization = trial_data.json()['parameterization']
trial_index = trial_data.json()['trial_index']

In [103]:
parameterization

{'x1': 0.12952612361032356,
 'x2': 0.3161371828308319,
 'x3': 0.4356693753712687,
 'x4': 0.03836543717822905,
 'x5': 0.5647218478354402,
 'x6': 0.7979930295270102}

In [107]:
results = evaluate(parameterization)

In [108]:
results

{'hartmann6': (-0.8447711151439348, 0.0)}

In [109]:
result_data = {'trial_index':trial_index, 'results':results}

In [110]:
requests.get('http://127.0.0.1:5000/complete_trial', json = result_data)

<Response [200]>